In [1]:
import math
import numpy as np
import tensorflow as tf
import keras
from keras.models import Model
from pathlib import Path

TARGET_SIZE = (224, 224)
BATCH_SIZE = 32
DESIRED_TRAIN_SIZE = 100

# ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator


img_path = Path("training")  # Assuming 'images' is your directory


train_ds: tf.data.Dataset = keras.preprocessing.image_dataset_from_directory(
    img_path,
    labels=None,
    seed=123,
    image_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
)
img_count = len(train_ds.file_paths)
factor = math.ceil(DESIRED_TRAIN_SIZE / img_count)


data_augmentation = keras.Sequential(
    [
        keras.layers.experimental.preprocessing.Rescaling(1.0 / 255),
        keras.layers.experimental.preprocessing.RandomRotation(0.1),
        keras.layers.experimental.preprocessing.RandomWidth(0.1),
        keras.layers.experimental.preprocessing.RandomHeight(0.1),
        keras.layers.experimental.preprocessing.RandomZoom(0.1),
        keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        keras.layers.experimental.preprocessing.Resizing(*TARGET_SIZE),
    ]
)

train_ds = train_ds.repeat(factor).map(
    lambda x: (data_augmentation(x, training=True), data_augmentation(x, training=True))
)


print(train_ds.cardinality().numpy())

input_img = keras.layers.Input(shape=(224, 224, 3))
x = keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input_img)
x = keras.layers.MaxPooling2D((2, 2), padding="same")(x)
x = keras.layers.Conv2D(16, (3, 3), activation="relu", padding="same")(x)
encoded = keras.layers.MaxPooling2D((2, 2), padding="same")(x)

x = keras.layers.Conv2D(16, (3, 3), activation="relu", padding="same")(encoded)
x = keras.layers.UpSampling2D((2, 2))(x)
x = keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = keras.layers.UpSampling2D((2, 2))(x)
decoded = keras.layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")
steps_per_epoch = img_count * factor // BATCH_SIZE

autoencoder.fit(train_ds, epochs=1, steps_per_epoch=steps_per_epoch)

2024-05-01 21:43:08.624369: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-01 21:43:08.626360: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-01 21:43:08.658740: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 21:43:08.658768: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 21:43:08.658787: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

Found 1 files belonging to 1 classes.
100
3/3 [==============================] - 1s 54ms/step - loss: 0.6923


In [2]:
img_path = Path("collection")

collection_ds: tf.data.Dataset = keras.preprocessing.image_dataset_from_directory(
    img_path,
    labels=None,
    image_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
)


file_paths = [Path(path).name for path in collection_ds.file_paths]


data_augmentation = keras.Sequential(
    [
        keras.layers.Rescaling(1.0 / 255),
        keras.layers.Resizing(*TARGET_SIZE),
    ]
)

collection_ds = collection_ds.map(
    data_augmentation, num_parallel_calls=tf.data.AUTOTUNE
)


def compute_error(batch):
    reconstructions = autoencoder(batch, training=False)
    return tf.reduce_mean(tf.abs(batch - reconstructions), axis=(1, 2, 3))


errors = collection_ds.map(compute_error, num_parallel_calls=tf.data.AUTOTUNE)

errors = np.concatenate(list(errors.as_numpy_iterator()))

print(list(zip(file_paths, errors)))

Found 49 files belonging to 1 classes.
[('owl1.jpg', 0.4650032), ('pexels-dsd-143941-1829979.jpg', 0.22578938), ('pexels-george-desipris-792381.jpg', 0.3568748), ('pexels-pixabay-461329.jpg', 0.28034148), ('pexels-8moments-1181181.jpg', 0.20166254), ('pexels-ekamelev-751689.jpg', 0.27698615), ('pexels-frans-van-heerden-201846-631317.jpg', 0.19831905), ('pexels-alexasfotos-2220336.jpg', 0.17801252), ('pexels-itfeelslikefilm-1083502.jpg', 0.31200835), ('pexels-pixabay-70080.jpg', 0.18742986), ('pexels-dsd-143941-1829979 (1).jpg', 0.2404782), ('pexels-pixabay-45853.jpg', 0.34787196), ('pexels-pixabay-236599.jpg', 0.37365955), ('pexels-pixabay-2361.jpg', 0.120844625), ('pexels-pixabay-86596.jpg', 0.207748), ('pexels-pixabay-47547.jpg', 0.20994629), ('pexels-frans-van-heerden-201846-802112.jpg', 0.27385104), ('pexels-richard-segal-732340-1618606.jpg', 0.21831523), ('pexels-pixabay-64219.jpg', 0.20747449), ('pexels-belle-co-99483-847393.jpg', 0.23396052), ('pexels-jimbear-1000529.jpg', 0.280